# Function(s) to determine desired AB outcome

In [1]:
# Import statements
import pandas as pd

### Create df of desired outcomes indexed by baserunners and number of outs

In [2]:
def outcome_df() -> pd.DataFrame:
    '''
    Returns pandas dataframe of desired outcomes sorted by outs+baserunners.
    TODO: Account for balls and strikes, as well, maybe using multiindexing
    '''
    # GB=groundball, FB=flyball, Whiffs=swing+miss
    df = pd.DataFrame(data={'Outs':[0,1,2], 'First':['GB','GB','FB'], 'Second':['FB','FB','FB'], 
                                'Third':['Whiffs','Whiffs','FB'], 'First and Second':['GB','GB','FB'],
                                'First and Third':['Whiffs','GB','FB'], 'Second and Third':['Whiffs','Whiffs','FB'],
                                'Bases Loaded':['Whiffs','GB','FB'], 'Bases Empty':['FB','FB','FB']}
                              ).set_index('Outs')
    
    return(df)

### Create dataframe of best pitches to throw to achieve a desired outcome

In [3]:
def pitch_for_outcome() -> pd.DataFrame:
    '''
    Returns pandas dataframe of best pitches to throw sorted by desired outcome.
    NOTE: This is a hard-coded version of what we want to intelligently infer/predict using data!!!
    '''
    df = pd.DataFrame(data={'Outcome':['GB','FB','Whiffs'], 'Pitch':['Curveball','Fastball','Slider'],
                            'Location':['Low','Up','Low and Away']}).set_index('Outcome')
    
    return(df)

### Convert boolean baserunner columns from game_state dataframe into readable strings (good for keys and/or column names)

In [4]:
def to_baserunners(game_state_row: tuple) -> str:
    '''
    Converts baserunner info from row of game_state df to string key for later column matching.
    >>> to_baserunners((...,'1st':False,'2nd':True,'3rd':True,...))
    '2nd and 3rd'
    '''
    # Logic to translate baserunner info from game_state_row
    first = game_state_row.First
    second = game_state_row.Second
    third = game_state_row.Third
    
    if first:
        if second:
            if third:
                baserunners = 'Bases Loaded'
            else:
                baserunners = 'First and Second'
        elif third:
            baserunners = 'First and Third'
        else:
            baserunners = 'First'
    else:
        if second:
            if third:
                baserunners = 'Second and Third'
            else:
                baserunners = 'Second'
        elif third:
            baserunners = 'Third'
        else:
            baserunners = 'Bases Empty'
        
    # Return baserunner info string
    return(baserunners)

### Convert exact ball-strike count to type of count (batter ahead, even, first pitch, etc.)

In [5]:
def count_converter(balls:int = None, strikes:int = None) -> str:
    '''
    Converts ball-strike count to type/description of count. Takes two integers inputs, returns string.
    If no input given, returns 'All Counts'.
    Note: 'First Pitch' and 'Two Strikes' take priority over other categories. 'Full Count' not supported.
    >>> count_converter(2,2)
    'Even'
    >>> count_converter(0,0)
    'First Pitch'
    '''
    # Conversion logic
    if balls == 0 and strikes == 0:
        count_type = 'First Pitch'
    elif strikes == 2:
        count_type = 'Two Strikes'
    elif balls > strikes:
        count_type = 'Batter Ahead'
    elif strikes > balls:
        count_type = 'Pitcher Ahead'
    elif balls == strikes:
        count_type = 'Even'
    else:
        count_type = 'All Counts'
    
    # Return count_type string
    return(count_type)

### Select desired outcomes for each situation in a game_state dataframe

In [6]:
def desired_outcomes(game_state: pd.DataFrame) -> pd.Series:
    '''
    Returns pandas series of desired AB outcomes based on given game state dataframe.
    Return values may include 'GB','FB','Whiffs','BA' (groundball, flyball, swing+miss, min batting average).
    '''
    # Dataframe of desired outcomes sorted by outs+baserunners
    best_outcomes = outcome_df()
    
    # Loop through game_state observations, extract relevant info, match with desired outcome
    desired_outcomes = []
    for row in game_state.itertuples():
        outs = row.Outs
        baserunners = to_baserunners(row)
        desired_outcomes.append(best_outcomes.loc[outs,baserunners])  
        
    # Return desied outcomes as pandas series
    return(pd.Series(desired_outcomes).rename('Desired Outcome'))

### Choose pitch selection based on desired outcome

In [7]:
def desired_pitch(desired_outcomes: pd.Series) -> pd.DataFrame:
    '''
    Returns pandas dataframe of desired pitches to achieve desired outcomes in a given game state.
    Dataframe includes pitch type and location.
    '''
    # Dataframe of desired pitches sorted by desired outcome
    best_pitches = pitch_for_outcome()
    
    # Loop through desired outcomes and match with best pitch selection
    pitch_selections = pd.DataFrame(columns=best_pitches.columns)
    for outcome in desired_outcomes:
        pitch_selections = pitch_selections.append(best_pitches.loc[outcome],ignore_index=True)
        
    # Return pitch selections
    return(pitch_selections)

### Test case (change cell from markdown to code and run)
import project_path

import import_ipynb

from WhichPitch.Lib import sim_data


game_states = sim_data.sim_game_state(10)

outcomes = desired_outcomes(game_states)

pitches = desired_pitch(outcomes)

pd.concat([game_states,outcomes,pitches],axis=1)